In [97]:
import sys
import os

# Правильный путь с учетом вложенности
correct_path = "/mnt/c/Users/user/tensor-decompositions/tensor-decompositions"
sys.path.insert(0, correct_path)

# Проверка
print("Обновленный sys.path:")
print(sys.path[0])


Обновленный sys.path:
/mnt/c/Users/user/tensor-decompositions/tensor-decompositions


In [98]:
from tdecomp.matrix.decomposer import TwoSidedRandomSVD

In [99]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt
import logging
from datetime import datetime
from typing import List, Dict, Tuple

In [100]:
def setup_logging():
    os.makedirs("logs", exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = f"logs/experiment_{timestamp}.txt"
    
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

logger = setup_logging()

torch.manual_seed(42)

def save_results_to_txt(results: List[Dict], filename: str = None):
    """Сохраняет результаты в текстовый файл"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"logs/results_summary_{timestamp}_NORM.txt"
    
    with open(filename, "w") as f:
        f.write("=== EXPERIMENT RESULTS SUMMARY ===\n\n")
        for res in results:
            f.write(
                f"Matrix: {res['shape']}\n"
                f"  TwoSidedRSVD Error: {res['rsvd_error']:.4e}\n"
                f"  SVD Error: {res['svd_error']:.4e}\n"
                f"  RSVD Time: {res['rsvd_time']:.3f}s, SVD Time: {res['svd_time']:.3f}s\n"
                f"  RSVD Rank: {res['rsvd_rank']}, SVD Rank: {res['svd_rank']}\n\n"
            )

In [101]:
def run_single_experiment(
    matrix: torch.Tensor,
    decomposer: TwoSidedRandomSVD,
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
) -> Dict:
    X = matrix.to(device)
    
    # TwoSidedRSVD
    start_time = time.time()
    U_rsvd, S_rsvd, Vh_rsvd = decomposer.decompose(X)
    X_approx_rsvd = decomposer._two_sided_compose(U_rsvd, S_rsvd, Vh_rsvd)
    rsvd_time = time.time() - start_time
    
    rsvd_error = torch.norm(X - X_approx_rsvd).item()
    rsvd_relative_error = rsvd_error / torch.norm(X).item()
    
    # Torch SVD (baseline)
    start_time = time.time()
    U_svd, S_svd, Vh_svd = torch.linalg.svd(X, full_matrices=False)
    X_approx_svd = U_svd @ torch.diag(S_svd) @ Vh_svd
    svd_time = time.time() - start_time
    
    svd_error = torch.norm(X - X_approx_svd).item()
    svd_relative_error = svd_error / torch.norm(X).item()
    
    return {
        "shape": tuple(X.shape),
        "rsvd_error": rsvd_error,
        "rsvd_relative_error": rsvd_relative_error,
        "rsvd_time": rsvd_time,
        "rsvd_rank": len(S_rsvd),
        "svd_error": svd_error,
        "svd_relative_error": svd_relative_error,
        "svd_time": svd_time,
        "svd_rank": len(S_svd),
        "device": device
    }

In [102]:
def generate_matrix(shape: Tuple[int, int], ) -> torch.Tensor:
    return torch.randn(*shape)

In [103]:
def visualize_results(results: List[Dict]):
    square_results = [r for r in results if r["shape"][0] == r["shape"][1]]
    rect_results = [r for r in results if r["shape"][0] != r["shape"][1]]
    
    square_results.sort(key=lambda x: x["shape"][0])
    rect_results.sort(key=lambda x: x["shape"][1])  
    
    # Square matrix
    plt.figure(figsize=(12, 10))
    
    # Absolute error
    plt.subplot(2, 2, 1)
    plt.plot([r["shape"][0] for r in square_results], 
             [r["rsvd_error"] for r in square_results], "o-", label="RSVD")
    plt.plot([r["shape"][0] for r in square_results], 
             [r["svd_error"] for r in square_results], "o-", label="SVD")
    plt.xlabel("Matrix size (N x N)")
    plt.ylabel("Matrix norm error")
    plt.title("Square Matrices: Error vs Size")
    plt.legend()
    
    # Relative error
    plt.subplot(2, 2, 2)
    plt.plot([r["shape"][0] for r in square_results], 
             [r["rsvd_relative_error"] for r in square_results], "o-", label="RSVD")
    plt.plot([r["shape"][0] for r in square_results], 
             [r["svd_relative_error"] for r in square_results], "o-", label="SVD")
    plt.xlabel("Matrix size (N x N)")
    plt.ylabel("Relative error")
    plt.title("Square Matrices: Relative Error vs Size")
    plt.legend()
    
    plt.tight_layout()
    plt.savefig("visualizations/square_matrices_results_NORM.png")
    plt.close()
    
    # Rectangular matrix
    if rect_results:
        plt.figure(figsize=(12, 10))
        
        # Absolute error
        plt.subplot(2, 2, 1)
        plt.plot([f"{r['shape'][0]}x{r['shape'][1]}" for r in rect_results], 
                 [r["rsvd_error"] for r in rect_results], "o-", label="RSVD")
        plt.plot([f"{r['shape'][0]}x{r['shape'][1]}" for r in rect_results], 
                 [r["svd_error"] for r in rect_results], "o-", label="SVD")
        plt.xlabel("Matrix size (M x N)")
        plt.ylabel("Matrix norm error")
        plt.title("Rectangular Matrices: Error vs Size")
        plt.xticks(rotation=45)
        plt.legend()
        
        # Relative error
        plt.subplot(2, 2, 2)
        plt.plot([f"{r['shape'][0]}x{r['shape'][1]}" for r in rect_results], 
                 [r["rsvd_relative_error"] for r in rect_results], "o-", label="RSVD")
        plt.plot([f"{r['shape'][0]}x{r['shape'][1]}" for r in rect_results], 
                 [r["svd_relative_error"] for r in rect_results], "o-", label="SVD")
        plt.xlabel("Matrix size (M x N)")
        plt.ylabel("Relative error")
        plt.title("Rectangular Matrices: Relative Error vs Size")
        plt.xticks(rotation=45)
        plt.legend()
        
        plt.tight_layout()
        plt.savefig("visualizations/rectangular_matrices_results_NORM.png")
        plt.close()

In [104]:

def run_experiments():
    decomposer = TwoSidedRandomSVD(distortion_factor=0.6, random_init="normal")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    logger.info(f"Running experiments on {device}")
    
    square_sizes = [2**i for i in range(4, 13)]  
    rectangular_fixed_dim = 256
    rectangular_variable_sizes = [2**i for i in range(4, 13)]  
    
    results = []
    
    # Square matrix experiment
    logger.info("Starting square matrices experiments")
    for size in square_sizes:
        X = generate_matrix((size, size))
        result = run_single_experiment(X, decomposer, device)
        results.append(result)
        logger.info(
            f"Square {size}x{size}: "
            f"RSVD error={result['rsvd_error']:.2e}, "
            f"SVD error={result['svd_error']:.2e}"
        )
            
    
    # Rectangular matrix experiment
    logger.info("Starting rectangular matrices experiments")
    for size in rectangular_variable_sizes:
        X = generate_matrix((rectangular_fixed_dim, size))
        result = run_single_experiment(X, decomposer, device)
        results.append(result)
        logger.info(
            f"Rect {rectangular_fixed_dim}x{size}: "
            f"RSVD error={result['rsvd_error']:.2e}, "
            f"SVD error={result['svd_error']:.2e}"
        )
    
    save_results_to_txt(results)
    visualize_results(results)
    logger.info("All experiments completed")
    

In [106]:
run_experiments()

2025-08-05 15:51:25,931 - INFO - Running experiments on cuda
2025-08-05 15:51:25,932 - INFO - Starting square matrices experiments


TypeError: TwoSidedRandomSVD._two_sided_decompose() got an unexpected keyword argument 'epsilon'